In [1]:
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [3]:
USE_CUDA

True

In [4]:
def del_title(file):
    new = []
    with open(file, 'r', encoding='utf8') as f:
        lines = f.readlines()
    title_i = -1
    for i, line in enumerate(lines):
        if line[:4] == '<doc':
            title_i = i + 1
            continue
        if i == title_i:
            continue
        new.extend(line.splitlines())
    return ''.join(new)

In [5]:
h = del_title('wiki.txt')

In [6]:
t = re.sub(r'[^ ㄱ-ㅣ가-힣\.,"\?~\'!]+', 'a', h)

In [7]:
tt = list(t)
x = tt.copy()
for i, v in enumerate(tt):
    if v == '.':
        if tt[i-1] == '다':
            x[i] = '.\n'

xx = ''.join(x)
r = xx.split('\n')

y = []
for v in r:
    if 'a' in v:
        continue
    y.append(v)

c = []
for v in y:
    if v[0] == ' ':
        v = v[1:]
    if len(v) <= 10:
        continue
    c.append(v)

In [8]:
len(c)

1290835

In [9]:
text = '\n'.join(c)

In [10]:
len(text)

68213614

In [11]:
with open('corpus.txt', 'w', encoding='utf8') as f:
    f.write(text)

In [13]:
print(text[:990])

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
조지아 공과대학교를 졸업하였다.
조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다.
지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다.
인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다.
미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.
카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.
특히 국제 분쟁 조정을 위해 북한의 김일성, 아이티의 세드라스 장군, 팔레인스타인의 하마스, 보스니아의 세르비아계 정권 같이 미국 정부에 대해 협상을 거부하면서 사태의 위기를 초래한 인물 및 단체를 직접 만나 분쟁의 원인을 근본적으로 해결하기 위해 힘썼다.
이 과정에서 미국 행정부와 갈등을 보이기도 했지만, 전직 대통령의 권한과 재야 유명 인사들의 활약으로 해결해 나갔다.
또한 이 때 김영삼 대통령과 김일성 주석의 만남을 주선했다.
하지만 그로부터 수주일 후 김일성이 갑자기 사망하여 김일성과 김영삼의 정상회담은 이루어지지 못했다.
미국의 관타나모 수용소 문제, 세계의 인권문제에서도 관심이 깊어 유엔에 유엔인권고등판무관의 제도를 시행하도록 노력하여 독재자들의 인권 유린에 대해 제약을 하고, 국제형사재판소를 만드는 데 기여하여 독재자들 같은 인권유린범죄자를 재판소로 회부하여 국제적인 처벌을 받게 하는 등 인권 신장에 크나 큰 기여를 했다.

